In [7]:
import pandas as pd
import numpy as np

In [9]:
player_list = pd.read_csv("./data/proc/league_player_list_with_cost.csv").set_index("IDFANGRAPHS")

names = ["steamer", "depthcharts", "razzball", "ATC", "thebat", "ZiPS"]
av_names = []   
for n in names:
    try:
        bats = pd.read_csv("./data/proc/{}_bats.csv".format(n)).set_index("playerid")[["POINTS"]]
        pitch = pd.read_csv("./data/proc/{}_pitch.csv".format(n)).set_index("playerid")[["POINTS"]]
        ranks = pitch.add(bats, fill_value=0).sort_values("POINTS")
        player_list = player_list.join(ranks, how="inner").sort_values("POINTS")
        player_list.rename(columns={"POINTS":"POINTS_{}".format(n)}, inplace=True)
        av_names.append("POINTS_{}".format(n))
    except:
        print("{} data unavailable.".format(n))

razzball data unavailable.


In [10]:
# Calculate totals
player_list["MEAN"] = np.mean(player_list[av_names], axis=1)
player_list["STD"] = np.std(player_list[av_names], axis=1)
player_list.sort_values("MEAN", inplace=True, ascending=False)
player_list["Rank"] = list(range(1, len(player_list)+1))
player_list["Rank_round"] = np.floor( (player_list.Rank  - 1 )/ 12 ) +  1
player_list["ADP_round"] = np.floor( (player_list.ADP  - 1 )/ 12 ) +  1
player_list["Keper_Value"] = player_list.Cost -  player_list.Rank_round
player_list = player_list[player_list.Rank_round <= 30]
player_list_keper = player_list.dropna()

In [11]:
player_list_keper.Status.value_counts()

99 prob     32
sfernand    29
Canha Ge    29
Pinstrip    28
miche       27
CALIXTO     27
FA          26
Hit         26
SLG         25
METS        23
Scavenge    23
lasmimis    22
La Maqui    21
Name: Status, dtype: int64

In [12]:
player_list_keper[player_list_keper.Status == "Canha Ge"].sort_values("Keper_Value", ascending=False).head(50)

,ID,Player,Position,Status,Age,ADP,Cost,POINTS_steamer,POINTS_depthcharts,POINTS_ATC,POINTS_thebat,POINTS_ZiPS,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value
20043,*047l7*,Luis Robert,OF,Canha Ge,24,23.46,30.0,59.006920,61.011115,59.564888,57.420929,53.777231,58.156217,2.472839,47,4.0,2.0,26.0
15711,*02yk4*,Tyler O'Neill,OF,Canha Ge,26,68.58,30.0,51.708488,51.975918,49.214180,48.843594,61.622715,52.672979,4.650621,95,8.0,6.0,22.0
16122,*03di6*,Jordan Romano,RP,Canha Ge,28,161.54,30.0,49.981390,49.718630,52.982145,52.882899,51.641913,51.441395,1.385033,110,10.0,14.0,20.0
12546,*02bq4*,C.J. Cron,1B,Canha Ge,32,157.06,26.0,53.140809,54.417567,53.160280,54.638583,54.294610,53.930370,0.646226,81,7.0,14.0,19.0
21390,*04ps7*,Joe Ryan,SP,Canha Ge,25,199.71,30.0,42.751956,42.468622,48.761392,44.372480,49.057716,45.482433,2.874188,179,15.0,17.0,15.0
19326,*03e5m*,Bryan Reynolds,OF,Canha Ge,26,88.29,17.0,57.849394,58.837083,62.251778,62.820635,63.625571,61.076892,2.295659,35,3.0,8.0,14.0
16358,*0312g*,Tyler Mahle,SP,Canha Ge,27,102.32,30.0,44.441611,44.261256,43.214414,52.157130,42.513805,45.317643,3.491334,181,16.0,9.0,14.0
21224,*0425t*,Bailey Ober,SP,Canha Ge,26,308.71,30.0,40.993457,40.699232,46.699885,42.389638,49.996844,44.155811,3.623749,203,17.0,26.0,13.0
16162,*02bu3*,Brandon Woodruff,SP,Canha Ge,28,17.29,15.0,60.352801,60.241098,64.500669,66.266154,61.265307,62.525206,2.425736,28,3.0,2.0,12.0
13394,*02hhd*,Dylan Floro,RP,Canha Ge,31,315.43,30.0,42.675719,42.518377,43.661556,46.692791,44.542545,44.018197,1.523634,207,18.0,27.0,12.0


In [13]:
player_list.to_csv('data/proc/league_player_list_with_ranks.csv', index=True, index_label='IDFANGRAPHS')

In [14]:
player_list_keper[(player_list_keper.Status == "miche") & (player_list_keper.Rank < 1000)].sort_values("Rank", ascending=True)

,ID,Player,Position,Status,Age,ADP,Cost,POINTS_steamer,POINTS_depthcharts,POINTS_ATC,POINTS_thebat,POINTS_ZiPS,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value
19755,*02yc4*,Shohei Ohtani,"UT,SP",miche,27,8.55,7.0,116.324263,115.324771,114.441623,121.518329,118.197064,117.161210,2.511040,1,1.0,1.0,6.0
10155,*01s8u*,Mike Trout,OF,miche,30,13.56,1.0,72.690386,72.809467,73.442893,78.741567,65.745468,72.685956,4.133498,10,1.0,2.0,0.0
15640,*02z0s*,Aaron Judge,OF,miche,29,35.24,5.0,68.123295,68.297998,67.628835,67.763056,66.834836,67.729604,0.507927,16,2.0,3.0,3.0
19361,*041rd*,Corbin Burnes,SP,miche,27,9.26,18.0,65.345240,65.256988,69.233865,68.289880,66.266372,66.878469,1.605903,18,2.0,1.0,16.0
18345,*03pir*,Kyle Tucker,OF,miche,24,15.40,13.0,61.774260,64.699163,66.212883,62.025589,73.953967,65.733173,4.433958,20,2.0,2.0,11.0
14212,*02n96*,Josh Hader,RP,miche,27,64.23,10.0,56.876515,56.450532,62.918201,61.676679,62.836970,60.151780,2.884970,38,4.0,6.0,6.0
14162,*02i2q*,Carlos Correa,SS,miche,27,80.04,6.0,58.997387,61.085872,57.405086,56.614976,62.502270,59.321118,2.206049,41,4.0,7.0,2.0
15474,*02i37*,Lucas Giolito,SP,miche,27,35.44,17.0,50.859672,50.706261,58.118387,67.640063,54.382923,56.341461,6.268289,59,5.0,3.0,12.0
7005,*02zif*,Ryan Pressly,RP,miche,33,128.86,10.0,52.837929,52.466077,56.554849,56.727566,56.010139,54.919312,1.870037,74,7.0,11.0,3.0
4314,*00031*,Joey Votto,1B,miche,38,150.27,15.0,54.625030,54.608039,55.220103,55.867771,52.334020,54.530993,1.191916,78,7.0,13.0,8.0


In [18]:
keper_list = ["19361", "18345", "19755",  "14212", "15640", "10155", "14161", "4314", "15890", "15474"]

In [20]:
print("Total {} potential keepers".format(len(keper_list)))
print(player_list_keper.loc[keper_list, :].Position.value_counts())
player_list_keper.loc[keper_list, :].sort_values("Cost", ascending=True)

Total 10 potential keepers
OF       4
RP       2
SP       2
UT,SP    1
1B       1
Name: Position, dtype: int64


,ID,Player,Position,Status,Age,ADP,Cost,POINTS_steamer,POINTS_depthcharts,POINTS_ATC,POINTS_thebat,POINTS_ZiPS,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value
10155,*01s8u*,Mike Trout,OF,miche,30,13.56,1.0,72.690386,72.809467,73.442893,78.741567,65.745468,72.685956,4.133498,10,1.0,2.0,0.0
15640,*02z0s*,Aaron Judge,OF,miche,29,35.24,5.0,68.123295,68.297998,67.628835,67.763056,66.834836,67.729604,0.507927,16,2.0,3.0,3.0
19755,*02yc4*,Shohei Ohtani,"UT,SP",miche,27,8.55,7.0,116.324263,115.324771,114.441623,121.518329,118.197064,117.161210,2.511040,1,1.0,1.0,6.0
14212,*02n96*,Josh Hader,RP,miche,27,64.23,10.0,56.876515,56.450532,62.918201,61.676679,62.836970,60.151780,2.884970,38,4.0,6.0,6.0
18345,*03pir*,Kyle Tucker,OF,miche,24,15.40,13.0,61.774260,64.699163,66.212883,62.025589,73.953967,65.733173,4.433958,20,2.0,2.0,11.0
4314,*00031*,Joey Votto,1B,miche,38,150.27,15.0,54.625030,54.608039,55.220103,55.867771,52.334020,54.530993,1.191916,78,7.0,13.0,8.0
15474,*02i37*,Lucas Giolito,SP,miche,27,35.44,17.0,50.859672,50.706261,58.118387,67.640063,54.382923,56.341461,6.268289,59,5.0,3.0,12.0
19361,*041rd*,Corbin Burnes,SP,miche,27,9.26,18.0,65.345240,65.256988,69.233865,68.289880,66.266372,66.878469,1.605903,18,2.0,1.0,16.0
14161,*02i2k*,Byron Buxton,OF,miche,28,64.51,24.0,51.457746,51.978854,51.185836,54.065571,43.832896,50.504181,3.485345,123,11.0,6.0,13.0
15890,*03ab0*,Luis Severino,RP,miche,27,150.28,27.0,43.261327,37.674639,45.322999,44.607204,48.751219,43.923478,3.611638,210,18.0,13.0,9.0


In [67]:
player_list_keper[(player_list)
]

,ID,Player,Position,Status,Age,ADP,Cost,POINTS_steamer,POINTS_depthcharts,MEAN,STD,Rank,Rank_round,Keper_Value,ADP_round
19755,*02yc4*,Shohei Ohtani,"UT,SP",miche,27,8.55,7.0,116.324263,115.324771,115.824517,0.499746,1,1.0,6.0,1.0
10155,*01s8u*,Mike Trout,OF,miche,30,13.56,1.0,72.690386,72.809467,72.749927,0.059541,6,1.0,0.0,2.0
15640,*02z0s*,Aaron Judge,OF,miche,29,35.24,5.0,68.123295,68.297998,68.210646,0.087351,12,1.0,4.0,3.0
19361,*041rd*,Corbin Burnes,SP,miche,27,9.26,18.0,65.345240,65.256988,65.301114,0.044126,16,2.0,16.0,1.0
18345,*03pir*,Kyle Tucker,OF,miche,24,15.40,13.0,61.774260,64.699163,63.236712,1.462452,25,3.0,10.0,2.0
14162,*02i2q*,Carlos Correa,SS,miche,27,80.04,6.0,58.997387,61.085872,60.041630,1.044243,33,3.0,3.0,7.0
14212,*02n96*,Josh Hader,RP,miche,27,64.23,10.0,56.876515,56.450532,56.663524,0.212992,48,4.0,6.0,6.0
4314,*00031*,Joey Votto,1B,miche,38,150.27,15.0,54.625030,54.608039,54.616535,0.008495,61,6.0,9.0,13.0
5038,*01sjg*,Josh Donaldson,3B,miche,36,226.18,2.0,53.069815,53.256206,53.163011,0.093195,75,7.0,-5.0,19.0
13449,*02n3l*,Taylor Rogers,RP,miche,31,306.26,13.0,53.013190,52.586154,52.799672,0.213518,80,7.0,6.0,26.0
